In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from gensim.models import FastText
import numpy as np
np.random.seed(42)

In [5]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [7]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 7

In [8]:
#df7 = pd.read_csv('df7.csv')
df7a = pd.read_csv('df7a.csv', index_col=0)

In [9]:
df7a.head(2)

,account_balance,duration,credit_history,purpose,credit_amount,savings_bond_value,employed_since,intallment_rate,sex_marital,guarantor,residence_since,property,age,other_installment_plans,type_of_housing,nr_of_existing_credits,job,number_of_dependents,telephone,foreign,target,content
0,< 0,6,critical account/other credits existing (not a...,radio/television,1169,unknown/no savings account,>=7 years,4,male;single,bank,4,real estate,67,none,own,2,skilled employee;official,1,Yes,Yes,__label__1,__label__1 < 0 6 critical account/other credit...
1,0-200 DM,48,existing credits paid back duly till now,radio/television,5951,< 100 DM,1-4 years,2,female;divorced/separated/married,bank,2,real estate,22,none,own,1,skilled employee;official,1,No,Yes,__label__2,__label__2 0-200 DM 48 existing credits paid b...


# Models

In [10]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [11]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

# Fast Text 
### Linear Regression

## DF7a

In [12]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7a['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7a['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    X_train = X_train.apply(preprocess)
    X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7a',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.71
Mean Precision: 0.70
Mean Recall: 0.71
Mean F1-Score: 0.70


In [13]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF7a')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF7a,Fast Text - Linear Regression,None,1,0.773333,0.480000,0.816901,0.413793,0.794521,0.444444,71.0,29.0,58,17,12,13
1,DF7a,Fast Text - Linear Regression,None,2,0.769231,0.545455,0.857143,0.400000,0.810811,0.461538,70.0,30.0,60,18,12,10
2,DF7a,Fast Text - Linear Regression,None,3,0.743590,0.545455,0.852941,0.375000,0.794521,0.444444,68.0,32.0,58,20,12,10
3,DF7a,Fast Text - Linear Regression,None,4,0.797468,0.523810,0.863014,0.407407,0.828947,0.458333,73.0,27.0,63,16,11,10
4,DF7a,Fast Text - Linear Regression,None,5,0.740741,0.421053,0.845070,0.275862,0.789474,0.333333,71.0,29.0,60,21,8,11
5,DF7a,Fast Text - Linear Regression,None,6,0.697368,0.625000,0.854839,0.394737,0.768116,0.483871,62.0,38.0,53,23,15,9
6,DF7a,Fast Text - Linear Regression,None,7,0.783133,0.294118,0.844156,0.217391,0.812500,0.250000,77.0,23.0,65,18,5,12
7,DF7a,Fast Text - Linear Regression,None,8,0.783133,0.823529,0.955882,0.437500,0.860927,0.571429,68.0,32.0,65,18,14,3
8,DF7a,Fast Text - Linear Regression,None,9,0.807692,0.454545,0.840000,0.400000,0.823529,0.425532,75.0,25.0,63,15,10,12
9,DF7a,Fast Text - Linear Regression,None,10,0.720000,0.560000,0.830769,0.400000,0.771429,0.466667,65.0,35.0,54,21,14,11


### FastText Linear Regression with df7a data and imbalance data tackling (RandomUnderSampler)

In [14]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7a['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7a['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    X_train = X_train.apply(preprocess)
    X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7a',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.69
Mean Precision: 0.68
Mean Recall: 0.69
Mean F1-Score: 0.68


In [15]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF7a')].head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
10,DF7a,Fast Text - Linear Regression,Random Under Sampler,1,0.724138,0.724138,0.724138,0.724138,0.724138,0.724138,29.0,29.0,21,8,21,8
11,DF7a,Fast Text - Linear Regression,Random Under Sampler,2,0.629630,0.606061,0.566667,0.666667,0.596491,0.634921,30.0,30.0,17,10,20,13


### FastText Linear Regression with df7a data and imbalance data tackling (RandomOverSampler)

In [16]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7a['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7a['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    X_train = X_train.apply(preprocess)
    X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7a',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.68
Mean Precision: 0.68
Mean Recall: 0.68
Mean F1-Score: 0.67


In [17]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF7a')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
20,DF7a,Fast Text - Linear Regression,Random Over Sampler,1,0.714286,0.700000,0.689655,0.724138,0.701754,0.711864,29.0,29.0,20,8,21,9
21,DF7a,Fast Text - Linear Regression,Random Over Sampler,2,0.607143,0.593750,0.566667,0.633333,0.586207,0.612903,30.0,30.0,17,11,19,13
22,DF7a,Fast Text - Linear Regression,Random Over Sampler,3,0.666667,0.714286,0.750000,0.625000,0.705882,0.666667,32.0,32.0,24,12,20,8
23,DF7a,Fast Text - Linear Regression,Random Over Sampler,4,0.678571,0.692308,0.703704,0.666667,0.690909,0.679245,27.0,27.0,19,9,18,8
24,DF7a,Fast Text - Linear Regression,Random Over Sampler,5,0.636364,0.680000,0.724138,0.586207,0.677419,0.629630,29.0,29.0,21,12,17,8
25,DF7a,Fast Text - Linear Regression,Random Over Sampler,6,0.588235,0.571429,0.526316,0.631579,0.555556,0.600000,38.0,38.0,20,14,24,18
26,DF7a,Fast Text - Linear Regression,Random Over Sampler,7,0.640000,0.666667,0.695652,0.608696,0.666667,0.636364,23.0,23.0,16,9,14,7
27,DF7a,Fast Text - Linear Regression,Random Over Sampler,8,0.833333,0.700000,0.625000,0.875000,0.714286,0.777778,32.0,32.0,20,4,28,12
28,DF7a,Fast Text - Linear Regression,Random Over Sampler,9,0.551724,0.571429,0.640000,0.480000,0.592593,0.521739,25.0,25.0,16,13,12,9
29,DF7a,Fast Text - Linear Regression,Random Over Sampler,10,0.666667,0.648649,0.628571,0.685714,0.647059,0.666667,35.0,35.0,22,11,24,13


## Performance metrics for the Linear Regression

In [18]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF7a,Fast Text - Linear Regression,None,1,0.773333,0.480000,0.816901,0.413793,0.794521,0.444444,71.0,29.0,58,17,12,13
1,DF7a,Fast Text - Linear Regression,None,2,0.769231,0.545455,0.857143,0.400000,0.810811,0.461538,70.0,30.0,60,18,12,10
2,DF7a,Fast Text - Linear Regression,None,3,0.743590,0.545455,0.852941,0.375000,0.794521,0.444444,68.0,32.0,58,20,12,10
3,DF7a,Fast Text - Linear Regression,None,4,0.797468,0.523810,0.863014,0.407407,0.828947,0.458333,73.0,27.0,63,16,11,10
4,DF7a,Fast Text - Linear Regression,None,5,0.740741,0.421053,0.845070,0.275862,0.789474,0.333333,71.0,29.0,60,21,8,11
5,DF7a,Fast Text - Linear Regression,None,6,0.697368,0.625000,0.854839,0.394737,0.768116,0.483871,62.0,38.0,53,23,15,9
6,DF7a,Fast Text - Linear Regression,None,7,0.783133,0.294118,0.844156,0.217391,0.812500,0.250000,77.0,23.0,65,18,5,12
7,DF7a,Fast Text - Linear Regression,None,8,0.783133,0.823529,0.955882,0.437500,0.860927,0.571429,68.0,32.0,65,18,14,3
8,DF7a,Fast Text - Linear Regression,None,9,0.807692,0.454545,0.840000,0.400000,0.823529,0.425532,75.0,25.0,63,15,10,12
9,DF7a,Fast Text - Linear Regression,None,10,0.720000,0.560000,0.830769,0.400000,0.771429,0.466667,65.0,35.0,54,21,14,11


In [19]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - Linear Regression2.csv')